This notebook, adapted from https://github.com/Ranlot/public-key-encryption.


In [42]:
!pip install egcd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 0) Some essential background


### Converting from base 10 to other bases

We only need to be able to convert from base from base 10 to base 2.

*(Incidentally, the ``converter`` below will also convert to other bases &#8804; 10)*


In [43]:
def eulidean_division(x, y):
    return x // y, x % y

def verif(a, base):
    return sum([int(val) * base ** int(pos) for pos, val in enumerate(str(a)[::-1])])

# positive integers and base <= 10 only
# (we don't include special symbols for higher bases)
def converter(a, base):
    q, r = eulidean_division(a, base)
    xs = [str(r), ]
    while q > 0:
        q, r = eulidean_division(q, base)
        xs.append(str(r))
    result = int(''.join(xs[::-1]))
    assert verif(result, base) == a
    return result

In [44]:
print(converter(194759, 2))

101111100011000111


### Euclid's extended algorithm

Instead of implementing Euclid's extended algorithm ourselves, we decide to rely on the `egcd` library for functionality and focus on a few interesting outcomes of the theorem:
- Bézout's identity
- Modular inverses

In [45]:
from egcd import egcd 

def euclidAlg(fst, sec):
    # egcd would not be difficult to implement but we focus on the resulting properties:
    # a) Bézout's identity
    # b) Modular inverses
    
    gcd, fst_inv, sec_inv = egcd(fst, sec)
        
    # a) Bézout's identity
    assert gcd == (fst * fst_inv) + (sec * sec_inv)
    
    # b) Modular inverses
    assert gcd == fst * fst_inv % sec
    assert gcd == sec * sec_inv % fst
    
    return {'gcd' : gcd, 'fst_inv' : fst_inv % sec, 'sec_inv' : sec_inv % fst}

In case that the input numbers $(a, b)$ are coprimes (a very common situation in this notebook), their greatest common divisor is $\text{gcd}(a, b) = 1$ and we have the following identities:

\begin{equation}
a \cdot a^{-1}_{\%b} + b \cdot b^{-1}_{\%a} = 1 \quad
\text{where} \quad  
\begin{pmatrix}
a \cdot a^{-1}_{\%b} \, \equiv 1 \,\,\text{mod}\,\,b \\
b \cdot b^{-1}_{\%a} \, \equiv 1 \,\,\text{mod}\,\,a
\end{pmatrix}
\end{equation}

Note that $\text{gcd}(a,b)$ and the modular inverses $a^{-1}_{\%b}$ and $b^{-1}_{\%a}$ are all computed efficiently and simultaneously with `egcd`. 

### Fast modular exponentiation

We are following a classic *square and multiply* method.  

(Numbers never exceed the modulus and the complexity only grows linearly with the number of bits of the modulus.)

In [46]:
def multiplyList(xs, m) :
    result = 1
    for x in xs:
        result = result * x % m
    return result % m

def FasterModularExponentiation(b, e, m):
    def helper(kmax):
        d = {}
        s = b % m
        d[0] = s
        for _ in range(kmax):
            s = (s * s) % m
            d[_ + 1] = s
        return d

    e2 = converter(e, 2)
    necessaryBits = [_[0] for _ in enumerate(reversed(str(e2))) if _[1] == '1']
    d = helper(max(necessaryBits))
    allVals = [d[_] for _ in necessaryBits]

    return multiplyList(allVals, m)

In [47]:
FasterModularExponentiation(14, 9, 138)

44

Quick check with small integers

In [48]:
14 ** 9 % 138

44

# 1) Real-world SSH keys

We start by using **ssh-keygen** to generate a pair of public / private keys into directory _ssh_key_example_.  By default the private key **ssh_key** (default name when generating the key) is in PEM (Privacy-Enhanced Mail) format but the public key is not.  Given the private key, one can generate a PEM public key by running:

```openssl rsa -in ssh_key -pubout -out ssh_key.pub.pem```

This way the contents of both keys can be parsed with openssl.

## 1.1) Public Key

The public key is composed of 2 integers:

---
- the modulus denoted as $n$,
- and the public exponent $e$
---

Both are represented below in hexadecimal representation.


In [49]:
!rm -f /content/key
!ssh-keygen -t rsa -f /content/key -P ""

Generating public/private rsa key pair.
Your identification has been saved in /content/key.
Your public key has been saved in /content/key.pub.
The key fingerprint is:
SHA256:2X8te1idFz1vN4oI0wZfluS4xaiArqGZmHtr8Jx+XAI root@f73581166046
The key's randomart image is:
+---[RSA 2048]----+
|                 |
|           .     |
|    .     * .   .|
|  E. . . = B   o.|
|  ..  . S *     *|
|.. .. .+ = .   +B|
|oBoo o  + . o ++=|
|*.* o    . . o.o.|
|.=oo          .. |
+----[SHA256]-----+


In [50]:
!openssl rsa -in /content/key -pubout -out /content/key.pub.pem

writing RSA key


In [51]:
!openssl asn1parse -inform pem -in /content/key.pub.pem > /content/pub.parsed.txt
!cat /content/pub.parsed.txt
parse1 = open("/content/pub.parsed.txt").read().split("\n")

    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl=2 l=   0 prim: NULL              
   19:d=1  hl=4 l= 271 prim: BIT STRING        


In [52]:
pos = list(map(lambda x:"BIT STRING" in x,parse1)).index(True)
pos = int(parse1[pos].split(':')[0])
pos

19

In [53]:
!openssl asn1parse -inform pem -in /content/key.pub.pem -strparse 19 > /content/pub.parsed2.txt
!cat /content/pub.parsed2.txt
parse2 = open("/content/pub.parsed2.txt").read().split("\n")

    0:d=0  hl=4 l= 266 cons: SEQUENCE          
    4:d=1  hl=4 l= 257 prim: INTEGER           :BFCE955413490079628695A85A796D46F23359C0636428C3D41FBB03F1C0120EA4B743210618EC4FABFBE88CCF8875ABD635AA14F4932EDE405AF38394F8D040C07C838096D1D1B80C245BDA11A00A5F9383491AF4AB6EF5A90CEB56CC4A5B263EB4FD565C2B895183D5FB513375441E86AB1DBB387C41B2ADFEB30DF2F3523B19847D8AF3F0A3F48576021F0A7477C1DD5AE8F4D63B935050BB47165BD9DA45A1B06F12C09801E079ECEE55FDBC78DB58BF83234C6D7E4C216A6D5D2FF4178967DF1EF5DB6F887BEB2BF207F5299C90ACCE45A124B5E27EFC43256F20C75CAEAA4E08D3BBDF777EF2EF3E8FAE3ED92FD0FA4A08767BF9200D9296FD73495C05
  265:d=1  hl=2 l=   3 prim: INTEGER           :010001


In [54]:
pem_public_n=int(parse2[1].split("prim: INTEGER")[1].split(":")[1], 16)

In [55]:
print('public modulus =', pem_public_n)

pem_public_e = int('010001', 16)

print('\npublic exponent =', pem_public_e)

public modulus = 24213386277466503360508339462379970399888815288924123812787448517857377576480421638923093741188377077731741800739435089150626211088316425526497023044968615527327993401565531696705348695587047117222664461525192229041802354705613562580026615139640486119860745660139418791360161111763787744642667919465025734737824288154768010686744573550075743615626871487928963139185592247948639765503852368753645295247971803569231924783353460407820259682617253095723178724218189726949129749098172324298184811183496058819938870178693255212969244340512060187638711523337929385073900232497333469806659071600865634984429437740960524622853

public exponent = 65537


By convention, the public exponent is fixed to a 65537.  This value is rather small when compared to the modulus and the notebook will have a section to below to discuss the order of magnitudes of the different numbers.  We will check later (when we have access to the private key) that this value is indeed coprime to $\phi(n)$ as should be for RSA to work)

## 1.2) Private key

Obviously, the private key contains more information which is composed of the following 5 integers:

---
*(repeated from public key)*
- the modulus $n$
- the public exponent $e$
---
- the private exponent $d$ used for decrytion
- the first random prime $p$
- the second random prime $q$
---
*(to be used for optimized decryption via Chinese remainder theorem)*
- the first optizmized exponent $d_p$
- the second optizmized exponent $d_q$
- modular inverse $q^{-1}_{\%p}$


In [56]:
!openssl asn1parse -inform pem -in /content/key  > /content/parsed.txt
!cat /content/parsed.txt
parse3 = open("/content/parsed.txt").read().split("\n")

    0:d=0  hl=4 l=1188 cons: SEQUENCE          
    4:d=1  hl=2 l=   1 prim: INTEGER           :00
    7:d=1  hl=4 l= 257 prim: INTEGER           :BFCE955413490079628695A85A796D46F23359C0636428C3D41FBB03F1C0120EA4B743210618EC4FABFBE88CCF8875ABD635AA14F4932EDE405AF38394F8D040C07C838096D1D1B80C245BDA11A00A5F9383491AF4AB6EF5A90CEB56CC4A5B263EB4FD565C2B895183D5FB513375441E86AB1DBB387C41B2ADFEB30DF2F3523B19847D8AF3F0A3F48576021F0A7477C1DD5AE8F4D63B935050BB47165BD9DA45A1B06F12C09801E079ECEE55FDBC78DB58BF83234C6D7E4C216A6D5D2FF4178967DF1EF5DB6F887BEB2BF207F5299C90ACCE45A124B5E27EFC43256F20C75CAEAA4E08D3BBDF777EF2EF3E8FAE3ED92FD0FA4A08767BF9200D9296FD73495C05
  268:d=1  hl=2 l=   3 prim: INTEGER           :010001
  273:d=1  hl=4 l= 257 prim: INTEGER           :832D70D9483B3414D03A451E178A561F2043D84D086C96A89F697808918C8F8CEE3FDBAC3934506188B8C6E98E81C0205E66E34463A449787CC0F72BF575594FFF94E886C81F0573FEF1DAD3C01C191C1691C22CAA4A989F701742D860ED357B698574815C95F5DB06F410660F7052BF21D0E38B886D0

In [57]:
def get_num(idx):
  return int(parse3[idx].split("prim: INTEGER")[1].split(":")[1], 16)

We start by verifying that $n$ are $e$ are indeed identical between the public and private keys:

In [58]:
pem_private_n = get_num(2)

pem_private_e = int('010001', 16)

In [59]:
assert pem_private_n == pem_public_n
assert pem_private_e == pem_public_e

The following three consist of the private exponent $d$ and the two underlying primes $p, q$ that divide the factorize the (public) modulus

In [60]:
pem_private_d = get_num(4)

pem_private_p = get_num(5)

pem_private_q = get_num(6)

print('p =', pem_private_p)
print('\nq =', pem_private_q)
print('\nd =', pem_private_d)

p = 176720805057869662661950182208145485363724083317644939961889527967735824751929937113039410531351344031514374630069080085438759584745349881667813619389111490276890989301083718311840922019371674593942236535085455666518614220245832480098989333250379864936660738484699015803029919021173237371022077578788073843899

q = 137014916096254181200468738556690420696141322152111320987868348002212680990053110231353915836171576520196690167965485313368971554952115784030448495641137048094444572647613291784485411588992047942415495915759481399897372475716295330569026399598079958664162182870078463318421708246250440702080237840574523600447

d = 165596256518047232421585407181185079160385215994761455881860053102657356967122232979564518451226673329724338808755698327787389933501599174286757567465483363069772493744231303870795189569999700755456771568735316065410779153800190043547823812071627665040859130145278070349200265680510967926915943485106415987407107507305340192896351839821832582274211938236706610233823

We can check that the modulus $n = pq$ is indeed the product of the private random primes.  In addition, we also check that the public exponent $e$ is coprime to Euler's totient function $\varphi(n)$. Since $p$ and $q$ are primes, we have $$\varphi(n) = \varphi(p)\cdot\varphi(q) = (p-1)\cdot(q-1)$$.

In [61]:
assert pem_public_n == pem_private_p * pem_private_q
assert 1 == euclidAlg(pem_private_e, (pem_private_p - 1) * (pem_private_q - 1))['gcd']

Therefore we can verify that the private exponent is indeed the inverse of the public exponent modulo $\varphi(n)$

\begin{equation}
e \cdot d \equiv 1 \,\, \text{mod} \,\, \varphi(n)
\end{equation}

In [62]:
assert 1 == pem_public_e * pem_private_d % ((pem_private_p - 1) * (pem_private_q - 1))

The other numbers contained in the private key are not strictly necessary for decryption.  They will come in handy when using the Chinese remainder theorem to optimize the decryption part (discussed further below in the notebook).

In [63]:
pem_crt_exponent_p = get_num(7)

pem_crt_exponent_q = get_num(8)

pem_crt_qInv = get_num(9)

In [64]:
from math import log10, log, floor

def approx10(x): # take a power an exponent in base 2 and approximate in base 10
    b10exp = floor(log10(x))
    prefactor = round(x / 10 ** b10exp, 3)
    return '%s x 10^{%s}' % (prefactor, b10exp)

## 3.2) Encryption

The cipher is determined by the remainder of the division of the plaintext $m$ raised to the public exponent $e$ by the public modulus $n$

\begin{equation}
c = m^e \,\,\text{mod}\,\, n
\end{equation}

We use the square and multiply modular exponentation function implemented earlier in the notebook

In [65]:
def encrypt(msgInt):
  return FasterModularExponentiation(msgInt, pem_public_e, pem_public_n)

encrypted = encrypt(42)
encrypted

8435571059162558296853476580940099900963673808345215005729815767050952738618420048470798466397440020670823291641753394454998885263551529822133648908330602604774876825275009997178641666416887683918090462422851442236925462390827632779581386213551867324628781582199533907126560636422442812782035060951370584995249777863503556718326470705404319870747933927004016357372596926758081830309973858829913768714294628130438785406100507748371143732390698822053572532305084400628491293759318595064392368773991703799579246820153486229605959114905628896496350579802510909810685720759268393692385545648948277535511966818235744140038


## 3.3) Decryption

Similar to the encryption, we will show first how to decode the cipher using the Pycrypto library and then verify that our custom modular exponentiation function reproduces the same result.  The message can be recovered by evaluating the remainder of the division of the cipher $c$ raise to the private exponent $d$ with the public modulus $n$:

\begin{equation}
c^d \,\,\text{mod}\,\, n \equiv m \,\,\text{mod}\,\,n
\end{equation}

The proof of why this is true is given in the accompanying PDF document.  Note that the private exponent $d$ is very large (on the order to the modulus $n$) since it is defined as the modular inverse of the public exponent $e$ which is itself quite small:

\begin{equation}
e \cdot d \equiv 1 \,\,\text{mod}\,\, \phi(n)
\end{equation}


#### The private exponent $d$

One of the strenghts of RSA is that it is not feasible to evaluate Euler's totient function $\phi(n)$ without having previous knowledge of $p$ and $q$.  In other words, calculating $\phi(n)$ is as hard as factorizing $n$ into its prime factors.  However, if we do know the values of $p, q$ it is very easy to calculate $\phi(n)$ as

\begin{align}
\phi(n) &= \phi(p) \cdot \phi(q) \\
&= (p-1) \cdot (q-1)
\end{align}

In [66]:
phiN = (pem_private_p - 1) * (pem_private_q - 1)

First of all, we shoud verify that $e$ and $\phi(n)$ are co-primes (actually it was already checked earlier but we can do it again...).  This is a necessary condition for the modular inverse of $e$ to exist (and be unique).

In [67]:
egcd_e_phiN = euclidAlg(pem_public_e, phiN)

assert egcd_e_phiN['gcd'] == 1

In this case, $d$ is also calculated using Euclid's extended theorem.

In [68]:
my_private_d = egcd_e_phiN['fst_inv']

print('e = %d' % pem_public_e)
print('\nd = ', my_private_d)
print('\nd ~ %s' % (approx10(my_private_d)))
print('\nd / n ~ %s' % approx10(my_private_d / pem_public_n))

e = 65537

d =  16559625651804723242158540718118507916038521599476145588186005310265735696712223297956451845122667332972433880875569832778738993350159917428675756746548336306977249374423130387079518956999970075545677156873531606541077915380019004354782381207162766504085913014527807034920026568051096792691594348510641598740710750730534019289635183982183258227421193823670661023382338715328088270042674288644339571496873512454465454100767172638340945447948606014940366317353749973499630945301533989316592136097090291660686179930419297002405718310059546806661939912195073244347235884503723119131203486007433998363101651732248053680637

d ~ 1.656 x 10^{616}

d / n ~ 6.839 x 10^{-1}


With $e = 65537$, we have a private exponent $d \approx 8.5 \cdot 10^{615} \approx 0.35 \, n$. Because this decryption exponent is so big, it is more efficient to optimize the decryption using the Chinese remainder theorem than to perform a straightforward exponentiation directly on $c$.  

We will show in a section below how to do this optimization, but for now in the sake of simplicity, let us go ahead and calculate $c^d \,\,\text{mod}\,\,n$ directly.

In [69]:
def decrypt(my_cipher):
  return FasterModularExponentiation(my_cipher, my_private_d, pem_public_n)

decrypted = decrypt(encrypted)
decrypted

42

Verify that the initial message is indeed recovered by all methods

In [70]:
assert 42 == decrypted

### Sanity check

As a sanity check, let us change the private exponent $d$ into a fake $d_\text{fake} = d - 1$ almost identical to the real one.  Notice that we are talking about a difference of 1 to numbers on the order of $10^{615}$ and yet, as expected, the decryption completely fails to recover the original message.

In [71]:
FasterModularExponentiation(encrypted, my_private_d - 1, pem_public_n)

1345974351910154022599217047004642029816938447318249009334315874178352113469939240361786353190341949521519802439355306546150725199173877154558307978036036415682168145050668047645198014336346829587817273280991746634060389052555157415959658151433071045398435375663559679769269064229543125060101105441940791064180462744554774390051933897465996375416752595025522821560068127094293132169539501407492998344706192551132053811516318795346788182864496684904075515484439509377946002199791878707049828208502565786640152701932271936336613489149616196314001852159037620442829012927841202574974601622190841306930988862500926478574

# 5) Partial homomorphic encryption in "textbook" RSA

In this section, we show how unpadded RSA is a partially homomorphic encryption scheme.  

Let us consider two integers $x_1$ and $x_2$ on which we wish to perform some computation.  Instead of doing the computation locally, we want to perform it on some unsecure data processor (cloud for example...) without revealing to the remote server anything about our data other than the list of instructions to execute.


Denoting by $\mathcal{E}$ the "textbook" RSA encryption function, the secret numbers $x_1$ and $x_2$ are encrypted into ciphers $c_1$ and $c_2$

\begin{align}
c_1 &= \mathcal{E}(x_1) \\
c_2 &= \mathcal{E}(x_2) 
\end{align}

In [72]:
x1 = 585
x2 = 117

In [73]:
c1 = encrypt(x1)
c2 = encrypt(x2)

In [74]:
print('x1 =', x1)
print('c1 =',hex(c1))

print('\nx2 =', x2)
print('c2 =', hex(c2))

x1 = 585
c1 = 0xa3fef2f9fb9196d2effd642b537180430f145d4453bc41532e0e07cec468810a5b833453bfbb96b9294eacb4cd9cc47d6d9711185b9f38db6e07cbadd9d4d4dd0e6775dd880aab42afe6aaff97cefdc19ab2229aaf05e58e521fb112cbd1467c285b7d43c553f4f12ad21f85af8d9b53b7b4bb5a7ef85a8fde1bc17554afbe7db1c0a2b674944409cd15681f2f784823d1a48a154d304d16e7ed1166d42a5be9125744a1f5abc45e8072428695f29e718c4d28793b381b8d2860f5deb445f7b03877ffef293a36cefa6b470d99f8526f8a3e6a4621b4242ade5ca2f0334cd335f55ba1ee8c5ba4d9df0e9bbc3750ca5f8efe4d2c5ee8ea635b52d082f0903f0a

x2 = 117
c2 = 0xa9b6bf8f2d45549949db63aeb90c133ba0e1dbbba4e4736cf08eb3b7aa0e6d9adfd73d2dbafdab4fa257b152b3ab42f9821a23ee3136ecea3f4ba794b6552d99b2d184e764242674b77a59bb0ec72f83ddf59bac30b681f6e2b88b1f26c13bcbd368d86877a21d71f8461f6adbf6c361dd793ae8e1a9c7a96e76a84ea98ee7f18c9bfafd73e259341c3661bb3a0ccf6ce6115cbad785483773561c8f7c95d824defd2bcd5c679a49670c15df8d24596ad663168877405d005e212ecbc1ced186072d9cd2640263149d3c3e1c4d36232138c37989d2f428e857806eb813f81eefa9abff

The encrypted ciphers $c_2$ and $c_2$ can now be sent to an untrusted environment tasked with excecuting arithmetic instructions on the ciphers.

## 5.1) Multiplication

In [75]:
c1Mulc2 = c1 * c2

print('c1Mulc2 = ', hex(c1Mulc2))

c1Mulc2 =  0x6cb8605eb47e6d06e942681a7216a22790ea997c0c03ab4b924be7e60d2195c8bae0960c81a4ea969b2c47acfe3bb0e5a7f4b4edc4441c6fd4b09eafc3900530f62d7c0ebf87e13aca9e43fb4b24fecb6e412f0ce4679d4e8a8f29daf7f274967fb9a49d93786f3ec69deb4dab2302c6d54ff2e0628719506e2325cfd32062239552efce482a8f3d3c160334f02e6a78aa5c116b7e1176dd88db33f2c83c2c190e097e7f687140abf22a287f08488bc033b61ffd44ff6fd0f71a1d94c738e4f18fe23ad7ff1074c6a8e2625c2b68e0e149e387f83282ccb6a12990544a6058c782402613f622eaf2a12e21a15ed66643acd7a46081b1476db6de9786446da64fc41be64dd72ed2e3f498ee8223935fa156872e61367abab42e409529692dcc947a8e39ff3923fcfb3d3806542606e0d41230a0985fd3467981c7443e6b3a44a1b8a4e85b1a7e54a2b2cf96b36637f06ea96cab0e346deecd2f4a23810b4e3114743ef8eec2e9383db87320125a4aeba16148414c701e030e70a8f741a6b9e33ecba8df545f447aa0018dffcc6499facc67b7ffc3c10ad405c2e1ff2d30873f4846fc1fc9f8dd77b19a6cd4e22d93e510fa9f9ea69a714abe5dd060379a3ae9bcf0b679ded51c48aa8430cd286ab0a524924873d212eb41785e890175e0695ff51ede319048da936a157dd386975

Send the result of $c_1 \cdot c_2$ back to the data owner who is the only one able to decrypt

In [76]:
decrypt(c1Mulc2)

68445

which turns out to be exactly $x_1 \cdot x_2$ even though these numbers were never communicated to anyone

In [77]:
print(x1 * x2)

68445


**Why does this happen?**

For a more in-depth derivation, we refer to the accomapnying PDF document.  In a nutshell, it is easy to show that RSA encryption, denoted as $\mathcal{E}$ satisfies the multiplicative property: 

$$c_1 \cdot c_2 = \mathcal{E}(x_1) \cdot \mathcal{E}(x_2) = \mathcal{E}(x_1 \cdot x_2)$$

so that the decryption function $\mathcal{D}$ applied to the product $c_1\cdot c_2$ of the ciphers yields the product $x_1 \cdot x_2$ of the original two numbers.

$$\mathcal{D} (c_1 \cdot c_2) = \mathcal{D} (\mathcal{E}(x_1 \cdot x_2)) = x_1 \cdot x_2 $$

Notice that the untrusted environment never learned anything about $x_1, x_2$ or even about the result of $x_1 \cdot x_2$ since all it ever was exposed to were the encrypted ciphers $c_1, c_2$ and $c_1 \cdot c_2$.  Nevertheless, thanks to multiplicatice homomorphicity of RSA, it was able to perform a useful computation.

## 5.3) Addition / subtraction

Unfortunately, these operations are not homomorphic under RSA and one needs to resort to different encryption schemes...

In [78]:
decrypt(c1 + c2)

9538867955764522699455129043842014192287268865131317821768565312716010371029050921215232029888551281749173626223467096994000204159286188715493314311504587367630986125942867303415636753956120047124072920006938331069682854419358034279042074079635364429460472037075798884789814992396487777894223138620521126344974027831374970108082033487460230692493432006566620428473495523288633931462048486802812403007385838415444216140703172795453807470566101406867606803044471612017760762914307703998213570992886884446745425998234245068816057890466810310216040058044641405134863493483286262073427220334610630513173344496728457008570

which is clearly very different from

In [79]:
x1 + x2

702